In [1]:
import numpy as np
from numpy import linalg as lg

In [2]:
def ParametriKamere(T):
    T0 = T[:, :-1]

    #resenje problema za linalg.qr
    if lg.det(T0) < 0:
        T = -T

    c1 = lg.det(T[:, 1:])
    c2 = -lg.det(np.array([T[:,0], T[:,2], T[:,3]]).T)
    c3 = lg.det(np.array([T[:,0], T[:,1], T[:,3]]).T)
    c4 = -lg.det(T[:, :-1])

    c1 = c1/c4
    c2 = c2/c4
    c3 = c3/c4

    C = np.array([c1, c2, c3])

    Q, R = lg.qr(lg.inv(T0))

    #resavanje problema za linalg.qr
    if R[0][0] < 0:
        R[0] = -R[0]
        Q[:,0] = -Q[:,0]
    if R[1][1] < 0:
        R[1] = -R[1]
        Q[:,1] = -Q[:,1]
    if R[2][2] < 0:
        R[2] = -R[2]
        Q[:,2] = -Q[:,2]

    A = Q.T
    K = T0.dot(Q)

    if K[2][2] != 1:
        K = K / K[2][2]

    return K, A, C

In [3]:
#poslednja cifra indeksa 105/2019 --> 5
n=5

In [4]:
T = np.array([[5, -1-2*n, 3, 18-3*n],
             [0, -1, 5, 21],
             [0, -1, 0, 1]])

In [5]:
K, A, C = ParametriKamere(T)
print("Parametar K:\n", K.round(5))
print("Parametar A:\n", A.round(5))
print("Parametar C:\n", C.round(5))
print("\n")

Parametar K:
 [[ 5.  3. 11.]
 [ 0.  5.  1.]
 [ 0.  0.  1.]]
Parametar A:
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]]
Parametar C:
 [ 4.  1. -4.]




In [6]:
M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50*n, 1])
M5 = np.array([30*n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])

In [7]:
M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])

In [8]:
originali = np.array([M1, M2, M3, M4, M5, M6])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

In [9]:
def CameraDLP(originali, projekcije):
    x = originali[0][0]
    y = originali[0][1]
    z = originali[0][2]
    t = originali[0][3]

    xp = projekcije[0][0]
    yp = projekcije[0][1]
    zp = projekcije[0][2]

    A = np.array([
        [0, 0, 0, 0, -zp*x, -zp*y, -zp*z, -zp*t, yp*x, yp*y, yp*z, yp*t],
        [zp*x, zp*y, zp*z, zp*t, 0, 0, 0, 0, -xp*x, -xp*y, -xp*z, -xp*t]
    ])

    for i in range(1, len(originali)):
        x = originali[i][0]
        y = originali[i][1]
        z = originali[i][2]
        t = originali[i][3]


        xp = projekcije[i][0]
        yp = projekcije[i][1]
        zp = projekcije[i][2]

        row1 = np.array([0, 0, 0, 0, -zp*x, -zp*y, -zp*z, -zp*t, yp*x, yp*y, yp*z, yp*t])
        row2 = np.array([zp*x, zp*y, zp*z, zp*t, 0, 0, 0, 0, -xp*x, -xp*y, -xp*z, -xp*t])

        A = np.vstack((A, row1))
        A = np.vstack((A, row2))

    U,S,V = np.linalg.svd(A)

    T = V[-1].reshape(3,4)
    
    return T

In [10]:
T = CameraDLP(originali, projekcije)
T = T / T[0][0] 
print("Matrica kamere T:\n", T)
print("\n")

Matrica kamere T:
 [[ 1.00000000e+00 -1.02231495e+00  4.54073651e+00 -1.30562534e+03]
 [ 1.62801576e+00 -1.04567521e+00  4.12295127e+00 -1.48624279e+03]
 [ 4.06741738e-03 -1.51186292e-03  1.08775247e-02 -4.14687646e+00]]




In [11]:
#koordinate izmerene metrom u svetskom koordinatnom sistemu (u milimetrima)
A1 = np.array([107, 197, 39, 1])
A2 = np.array([0,188,141,1])
A3 = np.array([126,220, 18, 1])
A4 = np.array([105, 314, 0, 1])
A5 = np.array([313, 160, 56, 1])
A6 = np.array([241, 70, 39, 1])


#piksel koordinate projekcije (slika je dimenzija 450x575)
A1p = np.array([212,378,1])
A2p = np.array([257,221,1])
A3p = np.array([229,381,1])
A4p = np.array([330,416,1])
A5p = np.array([39,516,1])
A6p = np.array([23,372,1])

In [12]:
moji_originali = np.array([A1, A2, A3, A4, A5, A6])
moje_projekcije = np.array([A1p, A2p, A3p, A4p, A5p, A6p])

In [13]:
moje_T = CameraDLP(moji_originali, moje_projekcije)
moje_T = moje_T / moje_T[0][0] 
print("Matrica kamere T:\n", moje_T)
print("\n")
moje_K, moje_A, moje_C = ParametriKamere(moje_T)
print("Parametar K:\n", moje_K.round(5))
print("\n")
print("Parametar A:\n", moje_A.round(5))
print("\n")
print("Parametar C:\n", moje_C.round(5))
print("\n")

Matrica kamere T:
 [[ 1.00000000e+00 -4.49419554e-01  4.63190856e-01 -2.38257396e+02]
 [ 3.46188314e+00  1.95394479e+00  5.14213742e+00 -1.31355300e+03]
 [ 8.19956559e-03  4.16136349e-03  8.59438713e-03 -2.99374516e+00]]


Parametar K:
 [[ 65.12583 -21.55499  65.08423]
 [  0.       84.18269 509.49337]
 [  0.       -0.        1.     ]]


Parametar A:
 [[ 0.34534 -0.93063  0.12113]
 [-0.67552 -0.1569   0.72046]
 [ 0.65147  0.33063  0.68284]]


Parametar C:
 [285.40249 122.26448  16.84609]


